<a href="https://colab.research.google.com/github/pea-sys/Til/blob/master/%E5%8A%B9%E6%9E%9C%E6%A4%9C%E8%A8%BC%E5%85%A5%E9%96%80_2_1%E3%83%A1%E3%83%BC%E3%83%AB%E3%83%9E%E3%83%BC%E3%82%B1%E3%83%86%E3%82%A3%E3%83%B3%E3%82%B0%E3%83%87%E3%83%BC%E3%82%BF%E3%81%AE%E5%88%86%E6%9E%90%EF%BC%88%E5%9B%9E%E5%B8%B0%E7%B7%A8%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

書籍「効果検証入門」の写経です。  
環境：GoogleColab  
言語：R  

In [1]:
# (2) ライブラリの読み出し
library("tidyverse")
library("broom")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.2     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
# (3) データの読み込み
email_data <- read_csv("http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv")

Parsed with column specification:
cols(
  recency = col_double(),
  history_segment = col_character(),
  history = col_double(),
  mens = col_double(),
  womens = col_double(),
  zip_code = col_character(),
  newbie = col_double(),
  channel = col_character(),
  segment = col_character(),
  visit = col_double(),
  conversion = col_double(),
  spend = col_double()
)



In [3]:
# (4) 女性向けメールが配信されたデータを削除したデータを作成
male_df <- email_data %>%
  filter(segment != "Womens E-Mail") %>% # 女性向けメールが配信されたデータを削除
  mutate(treatment = ifelse(segment == "Mens E-Mail", 1, 0)) # 介入を表すtreatment変数を追加

In [4]:
# (5) セレクションバイアスのあるデータを作成
## seedを固定
set.seed(1)

## 条件に反応するサンプルの量を半分にする
obs_rate_c <- 0.5
obs_rate_t <- 0.5

## バイアスのあるデータを作成
biased_data <- male_df %>%
  mutate(obs_rate_c =
           ifelse( (history > 300) | (recency < 6) |
                     (channel == "Multichannel"), obs_rate_c, 1),
         obs_rate_t =
           ifelse( (history > 300) | (recency < 6) |
                     (channel == "Multichannel"), 1, obs_rate_t),
         random_number = runif(n = NROW(male_df))) %>%
  filter( (treatment == 0 & random_number < obs_rate_c ) |
            (treatment == 1 & random_number < obs_rate_t) )

In [5]:
# (6) バイアスのあるデータでの回帰分析
## 回帰分析の実行
biased_reg <- lm(data = biased_data, formula = spend ~ treatment + history)

## 分析結果のレポート
summary(biased_reg)


Call:
lm(formula = spend ~ treatment + history, data = biased_data)

Residuals:
   Min     1Q Median     3Q    Max 
 -4.74  -1.46  -1.26  -0.48 497.74 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.3241996  0.1444390   2.245  0.02480 *  
treatment   0.9026109  0.1743057   5.178 2.25e-07 ***
history     0.0010927  0.0003366   3.246  0.00117 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 15.36 on 31860 degrees of freedom
Multiple R-squared:  0.001339,	Adjusted R-squared:  0.001276 
F-statistic: 21.35 on 2 and 31860 DF,  p-value: 5.406e-10


In [6]:
## 推定されたパラメーターの取り出し
biased_reg_coef <- tidy(biased_reg)
biased_reg_coef

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.324199564,0.144438990,2.244543,2.480430e-02
treatment,0.902610917,0.174305713,5.178321,2.252514e-07
history,0.001092682,0.000336606,3.246176,1.170872e-03


treatmentの推定結果は0.9026109であり、その検定におけるp値も2.25e-07と非常に小さい値なので、帰無仮説（メール送信の効果はない）を棄却出来る。

#共変量の追加による効果への作用

In [7]:
## 推定されたパラメーターの取り出し
biased_reg_coef <- tidy(biased_reg)

# (7) RCTデータでの回帰分析とバイアスのあるデータでの回帰分析の比較
## RCTデータでの単回帰
rct_reg <- lm(data = male_df, formula = spend ~ treatment)
rct_reg_coef <- tidy(rct_reg)
rct_reg_coef

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.6527894,0.1027070,6.355841,2.093808e-10
treatment,0.7698272,0.1452479,5.300090,1.163201e-07


In [8]:
## バイアスのあるデータでの単回帰
nonrct_reg <- lm(data = biased_data, formula = spend ~ treatment)
nonrct_reg_coef <- tidy(nonrct_reg)
nonrct_reg_coef

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.5483062,0.126891,4.321081,1.557365e-05
treatment,0.9794465,0.172717,5.670817,1.433467e-08


In [9]:
## バイアスのあるデータでの重回帰
nonrct_mreg <- lm(data = biased_data,
                  formula = spend ~ treatment + recency + channel + history)
nonrct_mreg_coef <- tidy(nonrct_mreg)
nonrct_mreg_coef

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.502412896,0.3793847254,1.32428341,1.854184e-01
treatment,0.846575728,0.1784759605,4.74335998,2.111119e-06
recency,-0.040266556,0.0259469894,-1.55187775,1.207014e-01
channelPhone,-0.001778911,0.3040193436,-0.00585131,9.953314e-01
channelWeb,0.226159585,0.3034664353,0.74525403,4.561237e-01
history,0.001029897,0.0003753754,2.74364500,6.079519e-03


共変量を加えることでtreatmentの値は0.84となり、セレクションバイアスの影響がより少ない結果を得られました。

#OVBの確認

In [10]:
# (8) OVBの確認
## (a) history抜きの回帰分析とパラメーターの取り出し
short_coef <- biased_data %>%
  lm(data = .,
     formula = spend ~ treatment + recency + channel) %>%
  tidy()

## aの結果から介入効果に関するパラメーターのみを取り出す
alpha_1 <- short_coef %>%
  filter(term == "treatment") %>%
  pull(estimate)

## (b) historyを追加した回帰分析とパラメーターの取り出し
long_coef <- biased_data %>%
  lm(data = .,
     formula = spend ~ treatment + recency + channel + history) %>%
  tidy()

## bの結果から介入とhistoryに関するパラメーターを取り出す
beta_1 <- long_coef %>% filter(term == "treatment") %>% pull(estimate)
beta_2 <- long_coef %>% filter(term == "history") %>% pull(estimate)

## (c) 脱落した変数と介入変数での回帰分析
omitted_coef <- biased_data %>%
  lm(data = ., formula = history ~ treatment + channel + recency) %>%
  tidy()
## cの結果から介入変数に関するパラメーターを取り出す
gamma_1 <- omitted_coef %>% filter(term == "treatment") %>% pull(estimate)

In [11]:
beta_2*gamma_1
alpha_1 - beta_1

# (9) OVBの確認(broomを利用した場合)
## broomの読み出し
library(broom)

## モデル式のベクトルを用意
formula_vec <- c(spend ~ treatment + recency + channel, # モデルA
               spend ~ treatment + recency + channel + history, # モデルB
               history ~ treatment + channel + recency) # モデルC

## formulaに名前を付ける
names(formula_vec) <- paste("reg", LETTERS[1:3], sep ="_")

[1] 0.02805398

[1] 0.02805398

In [12]:
## モデル式のデータフレーム化
models = formula_vec %>%
  enframe(name = "model_index", value = "formula")

In [13]:
## まとめて回帰分析を実行
df_models <- models %>%
  mutate(model = map(.x = formula, .f = lm, data = biased_data)) %>%
  mutate(lm_result = map(.x = model, .f = tidy))

## モデルの結果を整形
df_results <- df_models %>%
  mutate(formula = as.character(formula)) %>%
  select(formula, model_index, lm_result) %>%
  unnest(cols = lm_result)

## モデルA,B,Cでのtreatmentのパラメータを抜き出す
treatment_coef <- df_results %>%
  filter(term == "treatment") %>%
  pull(estimate)

## モデルBからhistoryのパラメータを抜き出す
history_coef <- df_results %>%
  filter(model_index == "reg_B",
         term == "history") %>%
  pull(estimate)

In [14]:
## OVBの確認
OVB <- history_coef*treatment_coef[3]
coef_gap <- treatment_coef[1] - treatment_coef[2]
OVB # beta_2*gamma_1
coef_gap # alpha_1 - beta_1

[1] 0.02805398

[1] 0.02805398

#2.2.7 Posting treatment bias

In [17]:
# (10) 入れてはいけない変数を入れてみる
#visitとtreatmentとの相関
cor_visit_treatment <- lm(data = biased_data,
                          formula = treatment ~ visit + channel + recency + history) %>%
  tidy()
  
cor_visit_treatment

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.7257798116,1.116859e-02,64.984028,0.000000e+00
visit,0.1439984370,7.612550e-03,18.915927,2.298683e-79
channelPhone,-0.0750691632,9.482184e-03,-7.916865,2.514358e-15
channelWeb,-0.0738276088,9.465020e-03,-7.800048,6.376566e-15
recency,-0.0291990673,7.951049e-04,-36.723539,3.357491e-289
history,0.0001090132,1.171462e-05,9.305735,1.413107e-20


In [18]:
# visitを入れた回帰分析を実行
bad_control_reg <- lm(data = biased_data,
                      formula = spend ~ treatment + channel + recency + history + visit) %>%
  tidy()

bad_control_reg

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-0.4375778549,0.3756177555,-1.1649552,2.440459e-01
treatment,0.2939911413,0.1770577952,1.6604247,9.683888e-02
channelPhone,0.1209780272,0.2999526157,0.4033238,6.867127e-01
channelWeb,0.1173436844,0.2994010598,0.3919281,6.951140e-01
recency,0.0098825815,0.0256534210,0.3852344,7.000663e-01
history,0.0005251979,0.0003707108,1.4167320,1.565711e-01
visit,7.1640214660,0.2419206812,29.6131006,3.850904e-190


メール配信の効果は0.847から0.294へ大きく低下。
メールの配信がもともとの購買傾向が弱いユーザのサイト来訪を増やしていることがあります。メールを配信されたグループは、メールが配信されなくてもサイトへ来訪するような購買傾向の強いユーザと、メールがあるからサイトへ来訪するような購買傾向の弱いユーザになります。一方でメールが配信さんなかったグループは、もともとの購買傾向が強いユーザのみがサイトへ来訪していることになります。
